In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/ALL_DATA_1.csv',encoding = 'cp949')
df2 = pd.DataFrame(df.groupby(['닉네임','제품명'])['평점'].mean())
df2 = df2.reset_index(inplace=False,drop=False)

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# n개 이상의 데이터 리뷰를 남긴 유저, 제품을 추출하는 함수
def preprocessing(data, n):
    min_id = data['닉네임'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    data = data[data['닉네임'].isin(min_id)]

    min_beer = data['제품명'].value_counts() >= n
    min_beer = min_beer[min_beer].index.to_list()
    data = data[data['제품명'].isin(min_beer)]

    return data

data = pd.DataFrame()

# 10번 반복합니다.

print("10개 이상의 제품을 구매한 사람의 데이터만 뽑아오기")
print("사용할 수 있는 데이터 개수 (알고리즘에 사용되는 데이터 개수)")
for i in range(1,10):
    data = preprocessing(df2, 10)
    print(data.shape)

10개 이상의 제품을 구매한 사람의 데이터만 뽑아오기
사용할 수 있는 데이터 개수 (알고리즘에 사용되는 데이터 개수)
(1364, 3)
(1364, 3)
(1364, 3)
(1364, 3)
(1364, 3)
(1364, 3)
(1364, 3)
(1364, 3)
(1364, 3)


In [4]:
ratings = data.copy()
# 피벗 테이블을 이용해 유저 - 아이디 매트릭스 구성
ratings_matrix = ratings.pivot_table('평점',index='닉네임',columns='제품명')
ratings_matrix.head()
# fillna 함수를 이용해 Nan 처리
rating_matrix = ratings_matrix.fillna(0)
rating_matrix

제품명,[가방 은행놀이] 마트 시장놀이 종이지폐 동전 카드 장난감 어린이집 유치원 생일 선물 답례품,고고다이노 장난감 렉스 비키 토모 핑 로키 스톰 안키 스피노 케루 플레오 페리 포키 볼카 케티 다니 벨리 주니 헬리,국민 아기 텐트 인디언 유아 어린이 아이 놀이 플레이 하우스 장난감집 아기집,꼬마버스 타요 버스 장난감 로기 가니 라니 누리 하트 패트 미니카,꼬마버스 타요 친구들 로기 가니 킨더 씨투 피넛 프랭크 앨리스 토토 러비 패트 누리 하트 맥스 크리스 포코 스피드 캐리와봉봉 장난감 미니카,나혼자산다 LED 비행기 장난감 360도 회전 부메랑 비행기,남아 여아 장난감 선물 트레져X 닌자골드 드래곤 에일리언 실험실 킹스헌터 해적선,너프건 19종 다트 리필 4종 모음 해즈브로 오버워치 엘리트 메가 좀비스트라이크 저격총,디폼 블럭 8mm 세트 패키지 도안 소근육 발달 놀이 조립 캐릭터 어린이 유아 만들기 장난감 툭툭 톡톡 블록 초등 학생 학교 아동 끼우기 유치원 어린이집 창의 사각 폼폼,레츠토이 트럭 잠수함 캐리어 미니카 자동차 장난감 세트,...,중장비 자동차 장난감 덤프트럭 포크레인 불도저 레미콘 남아 어린이날 선물 모래놀이 5살,지하철여행장난감/ 기차놀이/ 사운드불빛/ 선택가능,크리에이트 로드 레일카 4종 공룡 소방 엘레베이터 중장비 트랙 미니카 장난감,클립인형 6살 7살 여자아이 어린이 겨울왕국 장난감 4살 5살 여아 선물 공주 놀이 세트,타요 불도저 빌리 중장비 장난감 자동차 미니카,타요 중장비 장난감 자동차 맥스 크리스 포코 패트 미니카,탑브라이트 원목 자석낚시놀이 숫자학습 4살 5살 선물 유아 아기 낚시장난감,토이벨 미니카 MSZ세계명차시리즈 리얼사운드카 아이장난감자동차 생일선물BEST,트랜스포머 자동차 변신 로봇 RC 무선조종 범블비 오토봇 RC카 나혼자산다 장난감 선물,헬로카봇 스타피너 헬로카봇 아이언트 비트런 로드세이버 차탄이네윷놀이 펭토킹 호크블루 싼타페에이스 로드세이버 장난감선물
닉네임,,,,,,,,,,,,,,,,,,,,,
0119****,0.0,0.00,5.0,5.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
alsw****,0.0,0.00,0.0,0.0,5.000000,0.0,0.0,5.000000,4.666667,5.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0
ange****,5.0,0.00,5.0,0.0,5.000000,0.0,0.0,4.250000,4.750000,5.0,...,0.0,0.0,0.0,0.0,0.0,4.8,0.0,0.0,5.0,5.0
babo****,5.0,0.00,0.0,0.0,5.000000,0.0,0.0,4.666667,0.000000,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
blac****,0.0,0.00,4.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.0,...,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yell****,0.0,0.00,0.0,0.0,0.000000,0.0,5.0,3.500000,5.000000,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
yeon****,5.0,5.00,5.0,5.0,0.000000,0.0,4.0,4.500000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0
yoon****,3.0,5.00,4.0,5.0,4.583333,0.0,0.0,0.000000,5.000000,5.0,...,0.0,5.0,0.0,5.0,5.0,5.0,5.0,0.0,0.0,4.5


In [5]:
# 유사도 계산을 위해 트랜스 포즈
ratings_matrix_T = rating_matrix.T
ratings_matrix_T.head()

닉네임,0119****,alsw****,ange****,babo****,blac****,blue****,chan****,cher****,choi****,chun****,...,with****,wjdd****,wldu****,wlsd****,yang****,yell****,yeon****,yoon****,youn****,zzan****
제품명,,,,,,,,,,,,,,,,,,,,,
[가방 은행놀이] 마트 시장놀이 종이지폐 동전 카드 장난감 어린이집 유치원 생일 선물 답례품,0.0,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.00,0.0,0.0,5.0,3.000000,0.00,0.0
고고다이노 장난감 렉스 비키 토모 핑 로키 스톰 안키 스피노 케루 플레오 페리 포키 볼카 케티 다니 벨리 주니 헬리,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.75,0.0,...,4.838710,0.0,0.0,0.00,0.0,0.0,5.0,5.000000,4.75,0.0
국민 아기 텐트 인디언 유아 어린이 아이 놀이 플레이 하우스 장난감집 아기집,5.0,0.0,5.0,0.0,4.0,4.5,5.0,5.0,5.00,0.0,...,0.000000,0.0,0.0,5.00,5.0,0.0,5.0,4.000000,4.60,3.5
꼬마버스 타요 버스 장난감 로기 가니 라니 누리 하트 패트 미니카,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,4.75,0.0,0.0,5.0,5.000000,0.00,0.0
꼬마버스 타요 친구들 로기 가니 킨더 씨투 피넛 프랭크 앨리스 토토 러비 패트 누리 하트 맥스 크리스 포코 스피드 캐리와봉봉 장난감 미니카,0.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,4.75,0.0,...,4.666667,4.7,0.0,5.00,0.0,0.0,0.0,4.583333,5.00,0.0


In [6]:
# 아이템 - 유저 매트릭스로부터 코사인 유사도 구하기
item_sim = cosine_similarity(ratings_matrix_T,ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬에 제품명을 매핑해 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim,index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(40, 40)


제품명,[가방 은행놀이] 마트 시장놀이 종이지폐 동전 카드 장난감 어린이집 유치원 생일 선물 답례품,고고다이노 장난감 렉스 비키 토모 핑 로키 스톰 안키 스피노 케루 플레오 페리 포키 볼카 케티 다니 벨리 주니 헬리,국민 아기 텐트 인디언 유아 어린이 아이 놀이 플레이 하우스 장난감집 아기집,꼬마버스 타요 버스 장난감 로기 가니 라니 누리 하트 패트 미니카,꼬마버스 타요 친구들 로기 가니 킨더 씨투 피넛 프랭크 앨리스 토토 러비 패트 누리 하트 맥스 크리스 포코 스피드 캐리와봉봉 장난감 미니카,나혼자산다 LED 비행기 장난감 360도 회전 부메랑 비행기,남아 여아 장난감 선물 트레져X 닌자골드 드래곤 에일리언 실험실 킹스헌터 해적선,너프건 19종 다트 리필 4종 모음 해즈브로 오버워치 엘리트 메가 좀비스트라이크 저격총,디폼 블럭 8mm 세트 패키지 도안 소근육 발달 놀이 조립 캐릭터 어린이 유아 만들기 장난감 툭툭 톡톡 블록 초등 학생 학교 아동 끼우기 유치원 어린이집 창의 사각 폼폼,레츠토이 트럭 잠수함 캐리어 미니카 자동차 장난감 세트,...,중장비 자동차 장난감 덤프트럭 포크레인 불도저 레미콘 남아 어린이날 선물 모래놀이 5살,지하철여행장난감/ 기차놀이/ 사운드불빛/ 선택가능,크리에이트 로드 레일카 4종 공룡 소방 엘레베이터 중장비 트랙 미니카 장난감,클립인형 6살 7살 여자아이 어린이 겨울왕국 장난감 4살 5살 여아 선물 공주 놀이 세트,타요 불도저 빌리 중장비 장난감 자동차 미니카,타요 중장비 장난감 자동차 맥스 크리스 포코 패트 미니카,탑브라이트 원목 자석낚시놀이 숫자학습 4살 5살 선물 유아 아기 낚시장난감,토이벨 미니카 MSZ세계명차시리즈 리얼사운드카 아이장난감자동차 생일선물BEST,트랜스포머 자동차 변신 로봇 RC 무선조종 범블비 오토봇 RC카 나혼자산다 장난감 선물,헬로카봇 스타피너 헬로카봇 아이언트 비트런 로드세이버 차탄이네윷놀이 펭토킹 호크블루 싼타페에이스 로드세이버 장난감선물
제품명,,,,,,,,,,,,,,,,,,,,,
[가방 은행놀이] 마트 시장놀이 종이지폐 동전 카드 장난감 어린이집 유치원 생일 선물 답례품,1.000000,0.238820,0.411578,0.309502,0.337959,0.317621,0.051198,0.347942,0.354257,0.257268,...,0.108164,0.197300,0.131512,0.210693,0.215909,0.247839,0.264940,0.140351,0.416051,0.402200
고고다이노 장난감 렉스 비키 토모 핑 로키 스톰 안키 스피노 케루 플레오 페리 포키 볼카 케티 다니 벨리 주니 헬리,0.238820,1.000000,0.505175,0.374797,0.425815,0.395600,0.177968,0.444128,0.445026,0.402893,...,0.237761,0.172728,0.234396,0.335551,0.344243,0.361406,0.332948,0.286098,0.333417,0.502401
국민 아기 텐트 인디언 유아 어린이 아이 놀이 플레이 하우스 장난감집 아기집,0.411578,0.505175,1.000000,0.426375,0.447090,0.392179,0.360324,0.578588,0.571733,0.648295,...,0.315876,0.393999,0.380571,0.491542,0.413420,0.550787,0.356251,0.302204,0.440940,0.663746


In [7]:
ratings_matrix.columns  

Index(['[가방 은행놀이] 마트 시장놀이 종이지폐 동전 카드 장난감 어린이집 유치원 생일 선물 답례품',
       '고고다이노 장난감 렉스 비키 토모 핑 로키 스톰 안키 스피노 케루 플레오 페리 포키 볼카 케티 다니 벨리 주니 헬리',
       '국민 아기 텐트 인디언 유아 어린이 아이 놀이 플레이 하우스 장난감집 아기집',
       '꼬마버스 타요 버스 장난감 로기 가니 라니 누리 하트 패트 미니카',
       '꼬마버스 타요 친구들 로기 가니 킨더 씨투 피넛 프랭크 앨리스 토토 러비 패트 누리 하트 맥스 크리스 포코 스피드 캐리와봉봉 장난감 미니카',
       '나혼자산다 LED 비행기 장난감 360도 회전 부메랑 비행기',
       '남아 여아 장난감 선물 트레져X 닌자골드 드래곤 에일리언 실험실 킹스헌터 해적선',
       '너프건 19종 다트 리필 4종 모음 해즈브로 오버워치 엘리트 메가 좀비스트라이크 저격총',
       '디폼 블럭 8mm 세트 패키지 도안 소근육 발달 놀이 조립 캐릭터 어린이 유아 만들기 장난감 툭툭 톡톡 블록 초등 학생 학교 아동 끼우기 유치원 어린이집 창의 사각 폼폼',
       '레츠토이 트럭 잠수함 캐리어 미니카 자동차 장난감 세트', '마마스앤파파스 장난감 인형유모차',
       '말하는 따라쟁이 앵무새 인형 말따라하는 장난감', '메이크잇업 13종 디럭스, 유아 화장놀이 세트, 어린이 화장품 장난감',
       '멜로디카트 아이스크림카트 카트장난감 역할놀이', '미니 자동차 장난감 6종세트 남자아이선물 풀백기능 초저가할인',
       '미니특공대 슈퍼공룡파워2 케라루시 키오맥스 테고리오 테라새미 티라볼트 타르보스톰 스테고마그마 로봇 합체 아머봇 장난감',
       '베이비버스 무선조종 변신 구조버스 몬스터 경찰차 견인차 소방차 키키묘묘 구조대 장난감',
       '봉봉몬스터 물나오는 싱크대 아이와 주방놀이 설거지 장난감',
       '붐키즈 원목낚시놀이 아이들의 

In [8]:
# 유사도가 높은 제품 2개만 추출하기

# 본 제품 아래 유사도 점수가 높은 2개 제품 

item_sim_df['아기 유아 아동 리얼 미니 쇼핑 카트 장난감'].sort_values(ascending=False)[:4]

제품명
아기 유아 아동 리얼 미니 쇼핑 카트 장난감                  1.000000
말하는 따라쟁이 앵무새 인형 말따라하는 장난감                 0.678568
스모네오 레고듀플로호환 통큰블록 블럭장난감                   0.629402
메이크잇업 13종 디럭스, 유아 화장놀이 세트, 어린이 화장품 장난감    0.602948
Name: 아기 유아 아동 리얼 미니 쇼핑 카트 장난감, dtype: float64